# Qdrant Tools Registry:

This notebook demonstrates how to register and wrap the Qdrant search tool for agent use, by importing the implementation from the qdrant_tools folder.

In [1]:
# Ensure src/ is in sys.path so 'tools' can be imported
# %%
import sys, os
try:
    # ✅ Running from a Python script (.py file)
    TOOLS_PATH = os.path.abspath(os.path.join(os.path.dirname(__file__), ".."))
except NameError:
    # ✅ Running from a Jupyter notebook (__file__ is not defined)
    TOOLS_PATH = os.path.abspath(os.path.join(os.getcwd(), ".."))
SRC_PATH = os.path.join(TOOLS_PATH)


if SRC_PATH not in sys.path:
    sys.path.insert(0, SRC_PATH)
    print(f"✅ SRC path added: {SRC_PATH}")
else:
    print(f"🔁 SRC path already in sys.path: {SRC_PATH}")

✅ SRC path added: /home/prashant-agrawal/projects/netflix_talk2data/src


In [2]:
"""
Qdrant Tools Registry: Central place to register and wrap Qdrant-related tools for agent use.
"""
#from langchain_openai import OpenAIEmbeddings
from langchain.tools import Tool
from langchain.tools import StructuredTool
#from langchain_openai import OpenAIEmbeddings
from qdrant_client import QdrantClient
from langchain_huggingface import HuggingFaceEmbeddings

# Import the QdrantSearchTool implementation from the actual server tool script
from tools.qdrant_tools.qdrant_server_tool import QdrantSearchTool


🔁 SRC path already in sys.path: /home/prashant-agrawal/projects/netflix_talk2data/src
📌 Collection Name: indian_startups


In [3]:
from pydantic import BaseModel, Field

class QdrantSearchInput(BaseModel):
    query: str = Field(..., description="Enhanced natural‑language query")
    filters: dict | None = Field(None, description="Optional metadata filters (e.g. {'year_founded': {'gte':2015}}")
    k: int = Field(5, description="Number of top‑K results to return")

In [4]:
# Utility: get collection name from config/loader if needed
from utils.qdrant_client_loader import get_qdrant_collection_name


COLLECTION_NAME = get_qdrant_collection_name()

#embedding_model = OpenAIEmbeddings()
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


# Instantiate the tool
qdrant_search_tool_instance = QdrantSearchTool(
    host="localhost",
    port=6333,
    collection_name=COLLECTION_NAME,
    embedding_model=embeddings
)

#def wrapped_qdrant_search(inputs: QdrantSearchInput) -> list:
#    print(f"\n[DEBUG] Query: {input.query}")
#    print(f"[DEBUG] Filters: {input.filters}")
#    print(f"[DEBUG] Top K: {input.k}")
#    try:
#        results = qdrant_search_tool_instance.search(
#                query=input.query,
#                filters=input.filters, 
#                k=input.k
#            )
#        print(f"[DEBUG] Raw results: {results}")
#        return results
#    except Exception as e:
#        print(f"[ERROR] Qdrant search failed: {e}")
#        return []



In [5]:
# For LangChain agent compatibility
qdrant_search_tool = StructuredTool.from_function(
    name="qdrant_search",
    func=qdrant_search_tool_instance.search,
    argument_schema=QdrantSearchInput,
    description="""Semantic + metadata search over Qdrant.  
                    Embed a text query, apply optional filters, and return the top‑K matching documents.
                """
                )

In [11]:
# %% [markdown]
# # 1. Test the StructuredTool Directly

# 1️⃣ Build a QdrantSearchInput instance
test_input = {
    "query":"Top AI startups in India",
    "filters":{"product_categories": "saas"},
    "k":3
}
# 2️⃣ Call the StructuredTool directly
results = qdrant_search_tool.run(test_input)  
# .run will parse and dispatch to your QdrantSearchTool.search under the hood

# 3️⃣ Inspect the returned list of dicts
print("🔎 StructuredTool returned:", type(results), "with", len(results), "hits")
for hit in results:
    print(" •", hit["id"], hit["score"], hit["payload"].get("company_name"))


🔎 StructuredTool returned: <class 'list'> with 3 hits
 • 175 0.3595778 navi
 • 454 0.3536254 mobikwik
 • 256 0.33331898 boat


---

**This notebook only imports and registers the Qdrant search tool. All implementation logic lives in the qdrant_server_tool.py script.**